In [ ]:
import networkx as nx
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import matplotlib.font_manager
from scipy.stats import pearsonr
from scipy.stats import linregress
from matplotlib import pyplot as plt
import matplotlib as mpl
from pycirclize import Circos
matplotlib.font_manager.fontManager.addfont('/h/tianyi/TS_datasets_reversion/Cell_Press_plot/Arial.ttf')
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
mpl.rcParams['font.size'] = 8 
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.titlecolor'] = 'black'
plt.rcParams['legend.labelcolor'] = 'black'
plt.rcParams['axes.linewidth'] = 0.5

In [ ]:
## mean drug combination PRC and ROC

all_score_roc_mean_all=pd.read_csv('/h/tianyi/TS_datasets_reversion/sci_paper_plot_stas/github/figure_plot_script/figure6/all_score_roc_mean_all.txt',sep='\t')
all_score_prc_mean_all=pd.read_csv('/h/tianyi/TS_datasets_reversion/sci_paper_plot_stas/github/figure_plot_script/figure6/all_score_prc_mean_all.txt',sep='\t')

fig,axes=plt.subplots(1,2,figsize=(6.8,11/4))
plt.subplots_adjust(hspace=1)
plt.rcParams['axes.linewidth'] = 0.5
sns.barplot(x='method', y='ROC_AUC_values', data=all_score_roc_mean_all,ax=axes[1],color="#D83B4E")
axes[1].set_title('',fontsize=8, fontname='Arial')
axes[1].set_xlabel('')
axes[1].set_ylabel('Mean AUC of ROC',fontsize=8, fontname='Arial')
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].tick_params(labelrotation=270,axis='x')
axes[1].tick_params(axis='x',labelsize=8, width=0.5)
axes[1].tick_params(axis='y',labelsize=8, width=0.5)
sns.barplot(x='method', y='ROC_AUC_values', data=all_score_prc_mean_all,ax=axes[0],color="#3488BE")
axes[0].set_title('',fontsize=8, fontname='Arial')
axes[0].set_xlabel('')
axes[0].set_ylabel('Mean AUC of PRC',fontsize=8, fontname='Arial')
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)
axes[0].tick_params(labelrotation=270,axis='x')
axes[0].tick_params(axis='x',labelsize=8, width=0.5)
axes[0].tick_params(axis='y',labelsize=8, width=0.5)
plt.savefig(f'mean_ROC_PRC_add_coremethod_figsize_v4.pdf',dpi=300,bbox_inches='tight')


In [ ]:
## subnetwork distance prc and roc curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve,auc
######original 
drug_combination=pd.read_csv('/h/tianyi/TS_datasets_reversion/primary/predict_drug/validation_drug/combination_drug_nolevel_notissue_v2/drug_combination_all.txt',sep='\t')
drug_combination=drug_combination[['Drugbank_id','Drugbank_name','target_seed']].drop_duplicates()
network_distance_original=pd.read_csv('/h/tianyi/TS_datasets_reversion/primary/predict_drug/network_distance_CFX/close_distance_auc_DGIdb_cutoff15_screen_library_plot/close_distance_auc_merge_DGIdbcutoff15_screen_approve_cancer_library_FDAdb.txt',sep='\t')
network_distance_prc_auc_original=pd.read_csv('/h/tianyi/TS_datasets_reversion/primary/predict_drug/network_distance_CFX/close_distance_auc_DGIdb_cutoff15_screen_library_plot/PRC/close_distance_PRC_merge_DGIdbcutoff15_screen_approve_cancer_library_FDAdb_AUC_all.txt',sep='\t')
network_distance_roc_auc_original=pd.read_csv('/h/tianyi/TS_datasets_reversion/primary/predict_drug/network_distance_CFX/close_distance_auc_DGIdb_cutoff15_screen_library_plot/ROC/close_distance_ROC_merge_DGIdbcutoff15_screen_approve_cancer_library_FDAdb_AUC_all.txt',sep='\t')

network_distance_prc_auc_original_extract=network_distance_prc_auc_original[['tissue','seed','ROC_AUC_values']].drop_duplicates()
network_distance_roc_auc_original_extract=network_distance_roc_auc_original[['tissue','seed','ROC_AUC_values']].drop_duplicates()
network_distance_prc_auc_original_extract.columns=['tissue','seed','PRC_AUC_values']
network_distance_roc_auc_original_extract.columns=['tissue','seed','ROC_AUC_values']
network_distance_roc_prc_auc_original=pd.merge(network_distance_prc_auc_original_extract,network_distance_roc_auc_original_extract,on=['tissue','seed'])
seed_tissue_all_filter_original=network_distance_roc_prc_auc_original[(network_distance_roc_prc_auc_original['PRC_AUC_values']>0.3) & (network_distance_roc_prc_auc_original['ROC_AUC_values']>0.6)]
network_distance_prc_auc_original_filter=network_distance_prc_auc_original[(network_distance_prc_auc_original['tissue'].isin(seed_tissue_all_filter_original['tissue'].tolist())) & (network_distance_prc_auc_original['seed'].isin(seed_tissue_all_filter_original['seed'].tolist()))]
network_distance_roc_auc_original_filter=network_distance_roc_auc_original[(network_distance_roc_auc_original['tissue'].isin(seed_tissue_all_filter_original['tissue'].tolist())) & (network_distance_roc_auc_original['seed'].isin(seed_tissue_all_filter_original['seed'].tolist()))]

def network_distance_rank_label_prc(network_data,tissue,seed):
    approved_data=drug_combination[drug_combination['target_seed']==seed]    
    top_gene_drug_top_num_label=pd.merge(network_data,approved_data,left_on='drugbank_id',right_on='Drugbank_id',how='left')
    top_gene_drug_top_num_label_sort=top_gene_drug_top_num_label.sort_values('distance_score')[::-1]
    top_gene_drug_top_num_label_sort['rank_score']=np.arange(1,top_gene_drug_top_num_label_sort.shape[0]+1)
    top_gene_drug_top_num_label_sort['label']=1
    top_gene_drug_top_num_label_sort.loc[top_gene_drug_top_num_label_sort['Drugbank_name'].isna(),'label']=0
    return(top_gene_drug_top_num_label_sort)

def f_score_cutoff_fpr01(tissue,seed):
    try:
        os.mkdir(f'/h/tianyi/TS_datasets_reversion/primary/predict_drug/drug_prioritization/stas_data/{tissue}_{seed}_stas')
    except:
        pass
    close_net_distance_eg=network_distance_original[ (network_distance_original['tissue']==tissue) & (network_distance_original['seed']==seed)]
    close_net_distance_label_eg=network_distance_rank_label_prc(close_net_distance_eg,tissue,seed)
    score = np.array(close_net_distance_label_eg['rank_score'].tolist())
    label = np.array(close_net_distance_label_eg['label'].tolist())
    false_positive_rate, true_positive_rate, threshold_roc = roc_curve(label, score)
    auc_precision_recall = roc_auc_score(label, score)
    auc_precision_recall_round=round(auc_precision_recall,2)
    precision_all, recall_all, thresholds_pr = precision_recall_curve(label, score)
    f_score_all=2*precision_all*recall_all/(precision_all+recall_all)
    f_score_df=pd.DataFrame()
    f_score_df['thresholds']=[0]+thresholds_pr.tolist()
    f_score_df['precision']=precision_all
    f_score_df['recall']=recall_all
    f_score_df['f_score']=f_score_all
    f_score_df_max=f_score_df[f_score_df['f_score']==f_score_df['f_score'].max()]
    false_true_positive_rate=pd.DataFrame()
    false_true_positive_rate['thresholds']=threshold_roc
    false_true_positive_rate['false_positive_rate']=false_positive_rate
    false_true_positive_rate['true_positive_rate']=true_positive_rate
    max_thre=int(f_score_df_max['thresholds'].iloc[0])
    false_true_positive_rate_max=false_true_positive_rate[false_true_positive_rate['thresholds']==max_thre]
    while false_true_positive_rate_max.shape[0]==0:
        max_thre=max_thre+1
        false_true_positive_rate_max=false_true_positive_rate[false_true_positive_rate['thresholds']==max_thre]
    predict_true_drug_f_score=close_net_distance_label_eg[close_net_distance_label_eg['rank_score']>=max_thre]
    false_true_positive_rate_cutoff_thre=false_true_positive_rate[false_true_positive_rate['false_positive_rate']>=0.1].iloc[0,0]
    f_score_cutoff_df=f_score_df[f_score_df['thresholds']>=false_true_positive_rate_cutoff_thre]
    predict_true_drug_ftp=close_net_distance_label_eg[close_net_distance_label_eg['rank_score']>=false_true_positive_rate_cutoff_thre]
    predict_true_drug_ftp_drop_target=predict_true_drug_ftp[~(predict_true_drug_ftp['target'].str.contains(seed))]
    predict_true_drug_ftp_drop_target.to_excel(f'//h/tianyi/TS_datasets_reversion/primary/predict_drug/drug_prioritization/stas_data/candidate_validation_drug/{tissue}_{seed}_drug_prioritization.xlsx',index=False)
    Threhold_distance_score=round(predict_true_drug_ftp['distance_score'].iloc[0],2)
    F1_score=round(f_score_cutoff_df['f_score'].iloc[0],2)
    Precision=round(f_score_cutoff_df['precision'].iloc[0],2)
    Recall=round(f_score_cutoff_df['recall'].iloc[0],2)
    return(predict_true_drug_f_score,predict_true_drug_ftp,false_positive_rate,true_positive_rate,false_true_positive_rate_cutoff_thre,predict_true_drug_ftp,f_score_cutoff_df)

def original_ROC_AUC_plot_main(network_distance_original):
network_distance_prc_auc_original_mean_value=round(seed_tissue_all_filter_original['PRC_AUC_values'].mean(),2)
network_distance_roc_auc_original_mean_value=round(seed_tissue_all_filter_original['ROC_AUC_values'].mean(),2)
prc_all=pd.DataFrame()
roc_all=pd.DataFrame()
fig, axs = plt.subplots(1, 2, figsize=(6.8,11/4))
pos_all=range(seed_tissue_all_filter_original.shape[0])
plt.rcParams['axes.linewidth'] = 0.5
for pos in pos_all:
    seed_tissue_eg=seed_tissue_all_filter_original.iloc[pos:(pos+1),:]
    tissue=''.join(seed_tissue_eg['tissue'])
    seed_eg=''.join(seed_tissue_eg['seed'])
    ######prc######
    network_data_original=network_distance_original[(network_distance_original['tissue']==tissue) & (network_distance_original['seed']==seed_eg)]
    network_data=network_data_original.dropna()
    pcr_data_all=network_distance_rank_label_prc(network_data,tissue,seed_eg)
    score = np.array(pcr_data_all['rank_score'].tolist())
    label = np.array(pcr_data_all['label'].tolist())
    precision_original, recall_original, thres = precision_recall_curve(label, score)
    auc_precision_recall_original = auc(recall_original, precision_original)
    auc_precision_recall_round_original=round(auc_precision_recall_original,2)
    prc_eg=pd.DataFrame()
    prc_eg[f'precision_original_{tissue}_{seed_eg}']=precision_original
    prc_eg[f'recall_original_{tissue}_{seed_eg}']=recall_original
    prc_all=pd.concat([prc_eg,prc_all],axis=1)
    
    ########roc#######
    network_data=network_distance_original[(network_distance_original['tissue']==tissue) & (network_distance_original['seed']==seed_eg)]
    network_data=network_data.dropna()
    pcr_data_all=network_distance_rank_label_prc(network_data,tissue,seed_eg)
    score = np.array(pcr_data_all['rank_score'].tolist())
    label = np.array(pcr_data_all['label'].tolist())
    false_positive_rate_original, true_positive_rate_original, threshold_original = roc_curve(label, score)
    roc_original = roc_auc_score(label, score)
    roc_original=round(auc_precision_recall_original,2)
    network_data=network_distance_original[(network_distance_original['tissue']==tissue) & (network_distance_original['seed']==seed_eg)]
    network_data=network_data.dropna()
    pcr_data_all=network_distance_rank_label_prc(network_data,tissue,seed_eg)
    score = np.array(pcr_data_all['rank_score'].tolist())
    label = np.array(pcr_data_all['label'].tolist())
    false_positive_rate_original, true_positive_rate_original, threshold_original = roc_curve(label, score)
    roc_original = roc_auc_score(label, score)
    roc_original=round(auc_precision_recall_original,2)
    
    roc_eg=pd.DataFrame()
    roc_eg[f'false_positive_rate_original_{tissue}_{seed_eg}']=false_positive_rate_original
    roc_eg[f'true_positive_rate_original_{tissue}_{seed_eg}']=true_positive_rate_original
    roc_all=pd.concat([roc_eg,roc_all],axis=1)
    if ( tissue=='Skin') & (seed_eg=='BRAF'):
        axs[1].plot(recall_original, precision_original,color='#D83B4E', linewidth=1)
    elif  (( tissue=='Lung') & (seed_eg=='EGFR') ):
        axs[1].plot(recall_original, precision_original,color='#3488BE', linewidth=1)
    else:
        axs[1].plot(recall_original, precision_original,color='#deedd8', linewidth=0.5)
    axs[1].set_xlabel('Recall',fontsize=8, fontname='Arial')
    axs[1].set_ylabel('Precision',fontsize=8, fontname='Arial')
    axs[1].tick_params(axis='x',labelsize=8)
    axs[1].tick_params(axis='y',labelsize=8)
    axs[1].set_xlim(-0.1,1.1)
    axs[1].set_ylim(-0.1,1.1)
    axs[1].set_title('',fontsize=8, fontname='Arial')
    if (( tissue=='Skin') & (seed_eg=='BRAF') ):
        axs[0].plot(false_positive_rate_original, true_positive_rate_original,color='#D83B4E', linewidth=1)
    elif  (( tissue=='Lung') & (seed_eg=='EGFR') ):
        axs[0].plot(false_positive_rate_original, true_positive_rate_original,color='#3488BE', linewidth=1)
    else:
        axs[0].plot(false_positive_rate_original, true_positive_rate_original,color='#deedd8', linewidth=0.5)
    axs[0].set_xlabel('False positive rate',fontsize=8, fontname='Arial')
    axs[0].set_ylabel('True positive rate',fontsize=8, fontname='Arial')
    axs[0].tick_params(axis='x',labelsize=8)
    axs[0].tick_params(axis='y',labelsize=8)
    axs[0].set_xlim(-0.1,1.1)
    axs[0].set_ylim(-0.1,1.1)
    axs[0].set_title('',fontsize=8, fontname='Arial')
#axs[0].plot(false_positive_rate_original_all_matrix_mean.iloc[:,1], true_positive_rate_original_all_matrix_mean.iloc[:,1],color='#80AACA', linewidth=1.5)
#axs[1].plot(PRC_all_values_plot['recall_original_all_values_array'], PRC_all_values_plot['precision_original_all_array'],color='#86A5BC', linewidth=1.5)
axs[1].text(-0.25,1.15,f'D',fontsize=8, fontname='Arial')
axs[1].text(0.7,0,f'The mean AUC: {network_distance_prc_auc_original_mean_value}',fontsize=8, fontname='Arial')
axs[0].text(0.7,0,f'The mean AUC: {network_distance_roc_auc_original_mean_value}',fontsize=8, fontname='Arial')
axs[0].text(-0.25,1.15,f'C',fontsize=8, fontname='Arial')
plt.savefig(f'/h/tianyi/TS_datasets_reversion/sci_paper_plot_stas/data_statistic/drug_subnetwork_AUC_PRC/all_tissue_specific_subnetwork_ROC_PRC_skin_braf_highlight_mean_figsize_v2.pdf',dpi=300,bbox_inches='tight')


In [ ]:
####target pie plot#####
skin_braf=pd.read_excel('/h/tianyi/TS_datasets_reversion/Cell_Press_plot/drugcomb_validation_excel/Skin_BRAF_evidence.xlsx')
skin_braf_evidence_count=pd.DataFrame(skin_braf['Is it supported by literature'].value_counts())
skin_braf_evidence_count.columns=['evidence counts']
skin_braf_evidence_count['evidence type']=skin_braf_evidence_count.index.tolist()

lung_braf=pd.read_excel('/h/tianyi/TS_datasets_reversion/Cell_Press_plot/drugcomb_validation_excel/Lung_EGFR_evidence.xlsx')
lung_braf_evidence_count=pd.DataFrame(lung_braf['Is it supported by literature'].value_counts())
lung_braf_evidence_count.columns=['evidence counts']
lung_braf_evidence_count['evidence type']=lung_braf_evidence_count.index.tolist()

all_evidence=pd.read_excel('/h/tianyi/TS_datasets_reversion/Cell_Press_plot/drugcomb_validation_excel/judege_evidence_status_judge_unique_v2.xlsx')
unique_status_judge_stas=pd.DataFrame(all_evidence['unique_status_judge'].value_counts())
unique_status_judge_stas.columns=['evidence counts']
unique_status_judge_stas['evidence type']=unique_status_judge_stas.index.tolist()

fig, axes = plt.subplots(1,3,figsize=(6.8,11/4))
plt.rcParams['axes.linewidth'] = 0.5
axes[0].pie(skin_braf_evidence_count['evidence counts'],colors=['#EE7B6C','#7BC7B3','#80AACA'], 
autopct=lambda p: '{:.1f}%\n({:.0f})'.format(p, p * sum(skin_braf_evidence_count['evidence counts']) / 100),shadow=False,radius=1,startangle=140,pctdistance =0.75,textprops = {'fontsize': 8,'family':'Arial' })
axes[0].set_title('Skin BRAF subnetwork', fontsize=8, fontname='Arial')

axes[1].pie(lung_braf_evidence_count['evidence counts'],colors=['#EE7B6C','#7BC7B3','#80AACA'], 
autopct=lambda p: '{:.1f}%\n({:.0f})'.format(p, p * sum(lung_braf_evidence_count['evidence counts']) / 100),shadow=False,radius=1,startangle=140,pctdistance =0.75,textprops = {'fontsize': 8,'family':'Arial' })
axes[1].set_title('Lung EGFR subnetwork', fontsize=8, fontname='Arial')

axes[2].pie(unique_status_judge_stas['evidence counts'],colors=['#EE7B6C','#7BC7B3','#80AACA'], 
autopct=lambda p: '{:.1f}%\n({:.0f})'.format(p, p * sum(unique_status_judge_stas['evidence counts']) / 100),shadow=False,radius=1,startangle=140,pctdistance =0.75,textprops = {'fontsize': 8,'family':'Arial' })
axes[2].set_title('All subnetworks', fontsize=8, fontname='Arial')
plt.savefig(f'/h/tianyi/TS_datasets_reversion/sci_paper_plot_stas/data_statistic/publication_validation/all_subnetwork_publication_validation_figsize_v3.pdf',dpi=300,bbox_inches='tight')
